In [1]:
%pylab inline

import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout

Populating the interactive namespace from numpy and matplotlib


In [7]:
# Read data
train = pd.read_csv('../working/train_f.csv')
labels = train.ix[:,1].values.astype('int32')
X_train = (train.ix[:,2:].values).astype('float32')

In [64]:
# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(labels)

In [65]:
n = X_train.shape[1]
m_train = X_train.shape[0]

mu = numpy.mean(X_train, axis=0)
sigma = numpy.std(X_train, axis=0)

Xn_train = np.zeros(X_train.shape)

for i in range(n):
    if sigma[i] > np.spacing(1):
        Xn_train[:,i] = (X_train[:,i] - mu[i])/sigma[i]

In [84]:
input_dim = Xn_train.shape[1]
nb_classes = y_train.shape[1]
hidden_dim = 64
hidden_layers = 1
dropout = 0.15

In [85]:
model = None

# Here's a Deep Dumb MLP (DDMLP)
model = Sequential()
model.add(Dense(input_dim, hidden_dim, init='lecun_uniform'))
model.add(Activation('relu'))
model.add(Dropout(dropout))
for i in range(hidden_layers - 1):
    model.add(Dense(hidden_dim, hidden_dim, init='lecun_uniform'))
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
model.add(Dense(hidden_dim, nb_classes, init='lecun_uniform'))
model.add(Activation('relu'))

In [86]:
# we'll use MSE (mean squared error) for the loss, and RMSprop as the optimizer
model.compile(loss='mse', optimizer='rmsprop')

In [87]:
print("Training...")
model.fit(Xn_train, y_train, nb_epoch=15, batch_size=16, validation_split=0.1, show_accuracy=True, verbose=2)

Training...
Train on 8404 samples, validate on 2102 samples
Epoch 0
0s - loss: 0.0412 - acc.: 0.9626 - val. loss: 0.1082 - val. acc.: 0.8863
Epoch 1
0s - loss: 0.0364 - acc.: 0.9627 - val. loss: 0.1100 - val. acc.: 0.8863
Epoch 2
0s - loss: 0.0326 - acc.: 0.9665 - val. loss: 0.1080 - val. acc.: 0.8863
Epoch 3
0s - loss: 0.0339 - acc.: 0.9652 - val. loss: 0.1114 - val. acc.: 0.8863
Epoch 4
0s - loss: 0.0361 - acc.: 0.9626 - val. loss: 0.1124 - val. acc.: 0.8863
Epoch 5
0s - loss: 0.0342 - acc.: 0.9646 - val. loss: 0.1135 - val. acc.: 0.8863
Epoch 6
0s - loss: 0.0349 - acc.: 0.9639 - val. loss: 0.1122 - val. acc.: 0.8863
Epoch 7
0s - loss: 0.0359 - acc.: 0.9625 - val. loss: 0.1146 - val. acc.: 0.8863
Epoch 8
0s - loss: 0.0366 - acc.: 0.9617 - val. loss: 0.1193 - val. acc.: 0.8863
Epoch 9
0s - loss: 0.0359 - acc.: 0.9625 - val. loss: 0.1217 - val. acc.: 0.8863
Epoch 10
0s - loss: 0.0362 - acc.: 0.9621 - val. loss: 0.1276 - val. acc.: 0.8863
Epoch 11
0s - loss: 0.0351 - acc.: 0.9635 - val.

In [54]:
preds = model.predict_classes(Xn_train, verbose=0)
proba = model.predict_proba(Xn_train, verbose=0)

In [79]:
print Xn_train[0:10, :]

[[-1.04738855  1.00722623 -2.17247486  0.37116376 -0.00975668 -0.58797562
   1.1003902  -0.59398645 -0.090848   -0.02390458 -0.14692491]
 [-1.04738855  1.00722623 -2.17247486  0.37116376 -0.00975668 -0.58797562
  -0.90876859  1.68353975 -0.090848   -0.02390458 -0.14692491]
 [-0.71879941  1.36471558 -2.17247486 -0.6565991  -0.00975668 -0.58797562
  -0.90876859  1.68353975 -0.090848   -0.02390458 -0.14692491]
 [-1.29421377  1.17930448 -2.17247486 -0.42820737 -0.00975668 -0.58797562
   1.1003902  -0.59398645 -0.090848   -0.02390458 -0.14692491]
 [-1.29421377  1.17930448 -2.17247486 -0.42820737 -0.00975668 -0.58797562
  -0.90876859  1.68353975 -0.090848   -0.02390458 -0.14692491]
 [ 0.34662867  0.71370387 -2.17247486 -0.48530531 -0.00975668 -0.58797562
  -0.90876859  1.68353975 -0.090848   -0.02390458 -0.14692491]
 [ 0.47059566  0.44331706 -2.17247486 -0.6565991  -0.00975668 -0.58797562
  -0.90876859  1.68353975 -0.090848   -0.02390458 -0.14692491]
 [ 0.47336221  0.23034029 -2.17247486 -0.